# Entrega 2 - Clasificador (Árboles de Decisión)

### Grupo 07:
     - Renzo Gambone C.I. 5.155.486-4
     - Germán Ouviña C.I. 4.823.566-1
     - Leandro Rodríguez C.I 4.691.736-4


## 1. Introducción
***
### 1.1. Objetivo
***
El objetivo de esta tarea fue implementar un modelo de **árboles de decisión**, basándose su entrenamiento en el algoritmo **ID3** con una serie de modificaciones. Una vez generados distintos clasificadores en base a ciertos parámetros, se evaluó la performance de los mismos utilizando distintos tipos de métrica y comparando dichas evaluaciones para determinar cual modelo se ajustó mejor a cada escenario.

En términos formales, los parámetros del problema se reducen a lo siguiente:
- **Tarea *T*:** Clasificar ejemplos siguiendo cierto formato.
- **Experiencia *E*:** Conjunto de datos con ejemplos ya clasificados siguiendo cierto formato.
- **Performance *P*:** Múltiples tipos de medidas (en la sección 3 se profundizan).

### 1.2. Entrega
***
La entrega de esta tarea consta de dos grandes componentes:
- **Informe** en formato de Jupyter Notebook (este informe).
- **Programa** que permite entrenar clasificadores, evaluar su desempeño y clasificar nuevos ejemplos.

El objetivo del informe es centralizar la información relativa a la construcción del modelo así como los datos surgidos de las pruebas realizadas con distintas configuraciones paramétricas. Como agregado opcional, se adjuntan algunos scripts para ayudar a la lectura o probar en tiempo real ciertas funcionalidades.

Por otra parte, el programa ofrece una interfaz en consola que permite entrenar y evaluar árboles acorde a múltiples configuraciones paramétricas, mostrar los arboles entrenados y calificar elementos del conjunto asociado al árbol. También se agregó un script para entrenar y evaluar modelos automáticamente. Si bien todas estas herramientas fueron pensadas para uso del grupo, en el archivo README.md se adjunta una sencilla guía de como utilizarlas.

### 1.3. Formato
***
En las siguientes secciones se especifica el diseño del modelo, justificando la toma de decisiones a la hora de construirlo y se detallan las estrategias o algoritmos que fueron implementados para la configuración paramétrica, junto a las métricas utilizadas en la evaluación. Luego, se detalla la metodología de experimentación y con la misma los resultados obtenidos para cada modelo y cada conjunto de datos, habiendo entrenado con distintas configuraciones paramétricas. Finalmente se agregan conclusiones respecto a los resultados obtenidos. 

## 2. Diseño
***
En esta sección se detallan las características del diseño utilizado para construir el modelo, se profundizan las estrategias y algoritmos empleados en la configuración paramétrica y se tratan otros puntos como el procesamiento previo al entrenamiento y la evaluación posterior al mismo.

### 2.1. Modelo
***
La consigna propone la utilización y comparación de dos tipos de modelos que basan su entrenamiento en **ID3**. Dichos modelos serán denominados **árboles de decisión** y **bosques de decisión** respectivamente. A continuación, se especifican las características de cada uno:

**Notas:**
* No se ahonda en dichas descripciones puesto que son conceptos dados en el teórico.
* Se utiliza notación matemática para visualizar fácilmente la definición de cada modelo.

#### 2.1.1. Árbol de decisión
***
Para este escenario uno de los modelos propuestos es el de un **árbol de decisión** multiclase, es decir, una estructura de árbol que para un ejemplo dado, lo clasifica en una de múltiples clases.

Se definen más formalmente las siguientes nociones:
* $D =$ Conjunto de entrenamiento
* $C_D =$ Conjunto de posibles clasificaciones para $d \in D$
* $T_D =$ Árbol de decisión generado en base a $D$
* Se representa una clasificación de un ejemplo $d \in D$ realizada por el árbol $T_D$ como $T_D(d) = c : c \in C_D$ y $d \in D$

#### 2.1.2. Bosque de decisión
***

El otro modelo propuesto es el de un **bosque de decisión** multiclase de árboles binarios. El modelo de bosque se basa fuertemente en el modelo de árbol, consistiendo básicamente en un conjunto de árboles de decisión que clasifican un ejemplo para cada clase. El bosque pondera las clasificaciones de cada árbol en base a cierto criterio y genera una clasificación.

Se definen más formalmente las siguientes nociones:
* $D =$ Conjunto de entrenamiento
* $C_D =$ Conjunto de posibles clasificaciones para $d \in D$
* $B_D =$ Bosque de decisión generado en base a $D$
* $B_D(d) = c : c \in C_D ^ d \in D$, Representando esto una clasificación de un ejemplo $d \in D$ realizada por el bosque $B_D$
* $T_{(B, c)} =$ Árbol de decisión binario perteneciente a $B_D$ y generado en base a la clasificación $c \in C_D$
* Sea $|C_D| = n$, hay $n$ árboles de decisión binarios en $B_D$
* Se representa una clasificación de un ejemplo $d \in D$ realizada por el árbol $T_{(B, c)}$ como $T_{(B, c)}(d) = c' : c' \in \{True, False\}$ y $d \in D$

Los detalles sobre los algoritmos de entrenamiento y clasificación utilizados por cada modelo se expanden en la sección 2.3.



### 2.2. Preprocesamiento
***
Con los objetivos tanto de hacer viable, como de mejorar la performance de entrenamiento, se realizaron ciertos cambios en el conjunto de datos a utilizar. A continuación, se listan los cambios realizados agrupandolos en base a motivación y área cambiada en el conjunto original.

**Notas:**
* Los cambios realizados en el conjunto de datos fueron hechos en memoria, sin modificar de forma alguna los archivos originales.

#### 2.2.1. Optimización (Representación)
***
La elección de la representación interna fue compleja, ya que se tuvo en cuenta tanto la performance como la facilidad a la hora de programar e interpretar el código.

Luego de numerosas pruebas utilizando distintas representaciones, se determinó la utilización del objeto **DataFrame** de la biblioteca *pandas*. Al principio trajo muchos problemas de performance (tanto en uso de memoria como en velocidad) pero al implementar técnicas ofrecidas por la biblioteca dichos problemas se palearon, generando eventualmente un uso de memoria reducido y un entrenamiento veloz.

#### 2.2.2. Estructura (Atributos)
***
Originalmente se buscó mantener la estructura de atributos estática, con el fin de impactar lo menos posible en el entrenamiento y su posterior evaluación. No obstante, para el segundo conjunto de datos (en la sección 2.4 se expande este punto), fue necesario cambiar la estructura eliminando varios atributos y generando otros nuevos.

Más concretamente, el cambio realizado fue el proceso inverso al conocido como **one hot encoding**, ya que habia múltiples atributos que representaban "la idea" de un sólo atributo discreto. En múltiples problemas, la implementación de dicho algoritmo resulta de gran utilidad para mejorar el rendimiento tanto a nivel de performance como de clasificación, generando $x$ atributos binarios para $x$ posibles valores del atributo original. 

En el contexto de árboles de decisión, sucede lo opuesto: el aumento lineal en cantidad de atributos genera un aumento exponencial en el tamaño del árbol y en el tiempo de entrenamiento. Tomando esto en cuenta, se deshizo el **one hot encoding**, tratando a los atributos generados como continuos (se expande este punto y sus implicancias en las siguientes secciones).

#### 2.2.3. Clasificación (Resultados)
***
Para el modelo de **árbol de decisión** no fue necesario realizar ningún cambio en el conjunto de datos. No obstante, para entrenar a los árboles pertenecientes al **bosque de decisión**, al ser estos de carácter binario y funcionar para una sola clase, hubo que modificar los resultados del conjunto.

Utilizando las nociones de la sección 2.1, se procesó el conjunto de datos generando $|C_D| = n$ conjuntos con las clasificaciones correspondientes, uno para cada posible valor. Cada árbol del bosque fue entrenado utilizando uno de esos conjuntos generados. 

El conjunto original no fue modificado, por lo que el bosque en sí puede clasificar para las clases originales.

### 2.3. Algoritmo
***
En la siguiente sección se centralizan todas las especificaciones relativas a los algoritmos empleados, tanto para entrenar como para clasificar. 

**Notas:**
* Se hace referencia a puntos mencionados en secciones anteriores, sin entrar en detalle en lo que ya fue explicado.

#### 2.3.1. Algoritmo de entrenamiento
***
El algoritmo de entrenamiento se basa en el conocido **algoritmo ID3** para construcción de **árboles de decisión**. Dicho algoritmo sigue las pautas generales vistas en el teórico. Se agregó la posibilidad de tratar atributos continuos y se varió el uso de medidas para detectar el mejor atributo. Dichos cambios se tratan en las sub secciones 2.3.3 y 2.3.4. 

Teniendo en cuenta lo mencionado en secciones anteriores, el algoritmo de entrenamiento implementado puede entrenar dos tipos de modelos, generando dos tipos de clasificadores:

- **Árbol de decisión:** Se implementa el entrenamiento greedy estándar de ID3 con los cambios anteriormente mencionados y se obtiene como resultado un árbol de decisión multiclase.
- **Bosque de decisión:** Se itera sobre cada posible clasificación, procesando el conjunto de datos según corresponda y generando un árbol de decisión binario utilizando el algoritmo ID3 adaptado anteriormente mencionado.

#### 2.3.2. Algoritmo de clasificación
***
A la hora de clasificar un elemento como miembro de una clase, el procedimiento difiere dependiendo del modelo utilizado. No obstante, al igual que con el entrenamiento, la clasificación de un bosque se basa fuertemente en la de un árbol. 

Es importante mencionar que los árboles generados tienen una **probabilidad** asignada a cada hoja. Dicha probabilidad es calculada al momento de generar la hoja durante el entrenamiento. Al no quedar más atributos, se calcula la frecuencia de cada clase en el subconjunto de datos que hay en la hoja y aquella clasificación con mayor frecuencia es la elegida por la hoja, adjuntandose dicha frecuencia junto a ella.

Teniendo esto en cuenta, a grandes rasgos, los algoritmos funcionan como sigue:

- **Árbol de decisión:** Se toma el ejemplo a clasificar y se recorre recursivamente los nodos del árbol hasta llegar a una hoja. Se devuelve la clasificación junto a su probabilidad. Por la naturaleza del algoritmo, dicha probabilidad suele ser mayor a 0.5. 
- **Bosque de decisión:** Se toma el ejemplo a clasificar y se itera sobre cada árbol, obteniendo la clasificación del mismo. Una vez obtenidas todas las clasificaciones, se procede a realizar una **votación**. El resultado de dicha votación es la clasificación del ejemplo en cuestión.

La clasificación del **árbol de decisión** no cuenta con grandes especificidades, ya que es la misma utilizada por los árboles de decisión genéricos. No obstante, para determinar el algoritmo de votación del **bosque de decisión** fue necesario determinar un criterio, por lo que a continuación se explaya sobre este asunto.

En el escenario donde todos los árboles indican *False*, aludiendo a que el elemento no es de su clasificación y uno de los árboles indica *True*, la clasificación es trivial. En este caso el resultado final es la clasificación del elemento. Sin embargo este no es siempre el caso, dado que los árboles de decisión si bien son resistentes al ruido, no son inmunes al mismo.

Un problema a resolver a la hora de trabajar con bosques son los *empates*, en donde uno o más árboles devuelven *True* para determinado ejemplo. El desempate se alcanza comparando la **probabilidad** con la que cada árbol clasificó al ejemplo y tomando aquella clasificación con mayor frecuencia. 

En caso de que haya empate entre varias clasificaciones y sus probabilidades, el bosque elige una de ellas aleatoriamente. En la sección 4 se expanden las implicancias de este hecho.

#### 2.3.3. Selección de mejor atributo
***
Un paso crucial del algoritmo es decidir que atributo "bifurca mejor" el árbol. ID3 propone elegir el atributo que maximice la **ganancia de información**. Sin embargo, esta métrica favorece los atributos que toman múltiples valores sobre otros. A raiz de esto es que también se experimentó con otras estrategias en la configuración paramétrica, como el **ratio de ganancia** y la **reducción de "impureza"**

Todas las siguientes estrategias trabajan con una métrica que determina *"cuánto un atributo separa a los ejemplos según la clasificación objetivo"*, esta métrica es la **entropía** y se define como:
- Sea $D = $ Conjunto de datos
- Sea $C_D = $ Conjunto de posibles clasificaciones para $d \in D$
- Sea $|C_D| = n =$ Cantidad de posibles clasificaciones en $C_D$
- Sea $f_D : D \rightarrow C_D =$ Función que clasifica elementos en $D$
- Sea $p_i$ la proporción de ejemplos $d \in D : f_D(d) = c_i$ con $c_i \in C_D$ la i-ésima clasificación

$$Entropía(D) = -\sum p_i log_2(p_i)$$

La entropía mide la heterogeneidad de los datos: cuanto más homogéneos, menor será la misma.

A continuación, se adjunta una breve noción de las tres medidas utilizadas para determinar *"el mejor atributo"*:

##### 2.3.3.1. Ganancia
***
Se define la **ganancia de información** de un atributo $a$ sobre una muestra $D$ como:

$$Ganancia(D,a) = Entropía(D) - \sum_{v \in Val(a)}\frac{|D_v|}{|D|}\cdotp
 Entropía(D_v)\$$

La fundamentación de tomar la ganancia es que se desea que el atributo sobre el cual se particione divida el conjunto de datos de la forma más homogénea en cuanto a su clasificación posible.

Como se mencionó anteriormente, una desventaja de la ganancia es que favorece a los atributos que asumen un espectro muy alto de valores distintos con respecto a otros atributos. Un conjunto amplio y uniformemente distribuido de valores para el atributo no necesariamente es una característica que debería formar parte a la hora de determinar el mejor atributo para bifurcar.

##### 2.3.3.2. Ratio de ganancia
***
Se define el **ratio de ganancia** de un atributo $a$ sobre una muestra $D$, en función de la **separación de información** de un atributo $a$ sobre una muestra $D$. Dichas definiciones son las siguientes:

$$SeparacionDeInformacion(D,a) = - \sum_{v \in Val(a)}\frac{|D_v|}{|D|} \cdotp log_2(\frac{|D_v|}{|D|})$$

$$RatioGanancia(D,Aa) = \frac{Ganancia(D,a)}{SeparacionDeInformacion(D,a)}$$

La **separación de información** es sensible a que tan amplia y uniformemente el atributo distribuye sus datos, logrando así penalizar a los atributos que tomen un espectro muy amplio de valores. Una colección de ejemplos $D : |D| = n$ que estén completamente separados por un atributo $a_1$ (como una fecha) tendrá $SeparacionDeInformacion(D,a_1) = log_2(n)$. En cambio un atributo $a_2$ (booleano, por ejemplo) que divida los mismos $n$ ejemplos a la mitad tendrá $SeparacionDeInformacion(D,a_2) = 1$. Si ambos atributos producen la misma ganancia de información, entonces $a_2$ tendrá un **ratio de ganancia** más alto.

Se observa que la **separación de información** es la entropía de $D$ respecto a los valores del atributo $a$ (a diferencia de tomarla con respecto a la clasificación).

Una desventaja de usar el **ratio de ganancia** en lugar de sólo **ganancia** es que la **separación de información** puede ser 0, lo que hace el **ratio de ganancia** sea indefinido o muy grande para atributos que pueden tener el mismo valor para casi todos los miembros de $D$.

##### 2.3.3.3. Reducción de "impureza"
***
Se define la **reducción de impureza** de un atributo $a$ sobre una muestra $D$, utilizando la **impureza Gini** (nombrada en honor a Conrado Gini) de una muestra $D$. Dichas definiciones son las siguientes:

$$Gini(D) = 1 - \sum p_{i}^2$$

$$ReductionImpureza(D,a) = Gini(D) - \sum_{v \in Val(a)}\frac{|D_v|}{|D|} \cdotp Gini(D_v)$$

La **impureza Gini**, nombrada por el matemático Corrado Gini, mide la homogeneidad de un conjunto de elementos, tomando un valor mínimo de 0.0 y un valor máximo acotado por 1.0.

Dicha métrica toma valor 0 cuando todos los elementos del conjunto califican al mismo valor. En este contexto se dice que la **impureza** es mínima. A su vez si todos los valores del conjunto califican la misma cantidad de resultados distintos, se dice que la **impureza** es máxima (tomando un valor menor o igual a 1.0).

A los efectos practicos de la construcción de **árboles de decisión**, durante la misma se desea bifurcar con atributos que reduzcan la **impureza** lo mayor posible posible (optando por atributos que califiquen la mayor cantidad de instancias en la misma clase).

Una muy importante observación es que si bien la **impureza Gini** minimiza errores de clasificación y la **entropía** se usa para análisis exploratorio, ambas cumplen un rol casi idéntico. Usualmente, la **impureza Gini** da mejores resultados para atributos continuos mientras que la **entropía** funciona mejor con atributos discretos, sin embargo algunos estudios encontraron que sólo un 2% de las veces los resultados son distintos.

Debido a que su efectividad es dependiente del problema, se decidió incluir reducción de impureza como parte de la configuración paramétrica. 

#### 2.3.4. Interpretación de valores continuos
***
Una característica clave para el escenario propuesto y que **ID3** no contempla es **el tratamiento de atributos continuos**. Se emplearon tres estrategias distintas para interpretar los valores de tales atributos, teniendo como fin las mismas, discretizar el espectro continuo en una pequeña cantidad de clases.

Como noción general independiente a la estrategia implementada, cada clase discreta generada a partir de la estrategia en cuestión representa un **intervalo**. Los intervalos se definen por su **tope superior (máximo)**, excepto el último, que se define por su **tope inferior (mínimo)**

Sea por ejemplo, la lista de intervalos $[v_0, ..., v_n, v_{n+1}]$, para cada valor $v_i : i \in [0..n+1]$ se cumple lo siguiente:
- El intervalo de $v_0$ representa "todos los valores $k : k \leq v_0$".
- El intervalo de $v_i : i \in [1..n]$ representa "todos los valores $k : v_{i-1} \lt k \leq v_i$".
- El intervalo de $v_{n+1}$ representa "todos los valores $k : k \gt v_{n+1}$".

Es importante mencionar que los últimos dos **valores** cumplen ser iguales ($v_n = v_{n+1}$), lo que cambia es lo que el intervalo representa.

Habiendo visto esto, se resumen las estrategias implementadas:

##### 2.3.4.1. Partir en intervalos fijos
***
Como primera estrategia, se determina la siguiente:
- Sea $A_D =$ Conjunto de atributos en conjunto de datos $D$
- Sea $a \in A_D : Val(a) \subseteq N, Z, R$ un atributo continuo
- Sea $a^{*} \in A_D : |Val(a^{*})| \lt \infty$ la discretización de un atributo continuo
- Sea $\mu_{a}$ la mediana de $Val(a)$ para los ejemplos en $D$
- Entonces $Val(a^{*}) = \{\leq \mu_{a}, \gt \mu_{a}\}$

En lenguaje natural, se generan dos intervalos: los valores menores o iguales a la mediana de los valores actuales, y los valores mayores a dicha mediana. Todo nuevo valor entra siempre en alguno de los dos intervalos.

La fundamentación de tomar dos intervalos partidos por la mediana es que es una manera intuitiva de distribuir el atributo de forma "pareja" en el árbol. Por otra parte, la desventaja de esta estrategia está en que, además de no ser necesariamente el punto ideal de corte para la clasificación, el partir en solamente dos intervalos para atributos con múltiples valores puede terminar agrupando ejemplos e introduciendo ruido al conjunto de datos.

##### 2.3.4.2. Partir en intervalos variables
***
Con el fin de mejorar el defecto del enfoque anterior se implementa la siguiente estrategia:
- Sea $D_a =$ Conjunto de datos ordenado ascendentemente en base a atributo $a$
- Sea $A_{D_a} =$ Conjunto de atributos en conjunto de datos $D_a$
- Sea $a \in A_{D_a} : Val(a) \subseteq N, Z, R$ un atributo continuo
- Sea $a^{*} \in A_{D_a} : |Val(a^{*})| \lt \infty$ la discretización de un atributo continuo
- Sea $v_i : v_i \in Val(a)$ el valor de $a$ en el k-ésimo ejemplo $d_k \in D_a$ tal que cambia su clasificación respecto a $d_{k-1}$
- Entonces $Val(a^{*}) = \{\leq v_0, ..., \leq v_n, \gt v_n\}$

En lenguaje natural, se genera un número variable de intervalos. El procedimiento para generarlos es:
1. Ordenar $D$ en base a $a$ (generando $D_a$).
2. Iterar sobre $D_a$
3. Almacenar en $Val(a^{*})$, el valor $v_i$ de $a$ en $d_k$, en el momento en que la clasificación cambia del ejemplo $d_{k-1}$ al ejemplo $d_k$.

De esta forma, se tiene una cantidad de intervalos que relaciona el cambio en la clasificación del ejemplo con el cambio en el atributo $a$. Esta técnica impacta menos negativamente que la anterior en relación al ruido que introduce al recortar valores.

No obstante, en conjuntos de datos muy grandes con valores de $a$ muy dispersos o con clasificaciones muy cambiantes, puede suceder que se genere una cantidad de intervalos cercana a la cantidad de valores originales. Este hecho atenta en contra de la idea de interpretar valores continuos y de construir árboles simples con pocas ramas. No solamente eso, si no que también impacta en el tiempo de entrenamiento.

Teniendo esto en cuenta, se limitó la cantidad de posibles intervalos a 10, eligiendo 10 valores que se separan uniformemente en el conjunto total de intervalos.

##### 2.3.4.3. Partir por el valor que maximice la ganancia
***
Basado en el algortimo C4.5, el algortimo sucesor de ID3, se implementa la siguiente estrategia:
- Sea $D_a =$ Conjunto de datos ordenado ascendentemente en base a atributo $a$
- Sea $A_{D_a} =$ Conjunto de atributos en conjunto de datos $D_a$
- Sea $a \in A_{D_a} : Val(a) \subseteq N, Z, R$ un atributo continuo
- Sea $a^{*} \in A_{D_a} : |Val(a^{*})| \lt \infty$ la discretización de un atributo continuo
- Sea $v_i : v_i \in Val(a)$ el valor de $a$ en el k-ésimo ejemplo $d_k \in D_a$ tal que cambia su clasificación respecto a $d_{k-1}$
- Sea $Val(a') = \{\leq v_0, ..., \leq v_n, \gt v_n\}$ los posibles valores determinados en la sección anterior
- Entonces $Val(a^{*}) = \{\leq v_j, \gt v_j\} : v_j \in Val(a')$ y se maximiza la ganancia de partir el conjunto de datos en $v_j$

En lenguaje natural, se generan dos intervalos como en la sección 2.3.4.1 pero se parte de los valores tomados en la sección 2.3.4.2 y se determina cual de estos otorga una mayor ganancia.

Una desventaja de esto es que es de un mayor orden computacional, requiriendo una recorrida del dataset para calcular la ganancia de cada punto de corte a probar.


### 2.4. Evaluación
***
En la siguiente sección se centralizan todas las especificaciones relativas a la evaluación de los clasificadores entrenados.


#### 2.4.1. Conjuntos de evaluación
***
Los dos conjuntos de datos a partir de los cuales se entrenó y evaluó los modelos son:
- [**Conjunto *Iris*:**](https://archive.ics.uci.edu/ml/datasets/iris) El cual clasifica plantas del genero Iris (de la tribu Irideae, perteneciente a la familia Iridaceae) según especie. Cuenta con 150 ejemplos, 4 atributos de carácter continuo y 3 posibles clasificaciones.
- [**Conjunto *Covertype*:**](https://archive.ics.uci.edu/ml/datasets/Covertype) El cual clasifica terrenos de bosques según atributos cartográficos. Cuenta con 581012 ejemplos,54 atributos, siendo 10 de carácter continuo y 44 de carácter binario (son el one hot encoding de 2 atributos generando 4 y 40 atributos respectivamente) y 7 posibles clasificaciones.

Cabe destacar que el conjunto *Iris* cuenta con una cantidad ínfima de ejemplos, mientras que el conjunto *CoverType* va hacia el otro extremo. Las implicancias de estos hechos se tratan en la sección 3.

#### 2.4.2. Métodos de evaluación
***
Se experimentó con dos estrategias de evaluación.
- Validación 80/20
- Validación cruzada

Ambas estrategias subdividen el conjunto S de datos en dos conjuntos, E y T. E es usado para entrenar el modelo y T es usado para validar los datos.

**Hablar de que los subconjuntos deben de serguir la misma distribución que el original? Hablar de que estadisticamente Iris tiene un buen split y lo hace? Como es la onda con covertype??? No hablar de esto?**

##### 2.4.2.1. Validación 80/20
Para esta estratégia se agrupa en E el 80% de los datos de S. Tras entrenar el árbol se evalua con T (el otro 20% de S)

##### 2.4.2.2. Validación cruzada
Para esta estratégia se determina una cantidad de intervalos *n*. Luego S es partido en *n* intervalos.

Se entrena *n* veces, empleando el $intervalo_k$ para validar (T) y el resto de intervalos para entrenar (E). La medida se obtiene como el promedio de la medida en las *k* iteraciones.

Esta estrategia es mucho más robusta que la validación 80/20, sin embargo tiene el problema de tener un muy alto orden computacional, dado que tiene que correr *k* entrenamientos en lugar de uno.

En la experimentación se decidió fijar 10 como cantidad de intervalos.

**8? Usamos cruzada? Solo para iris?**

#### 2.4.3. Métricas de evaluación
***
Se utilizaron múltiples métricas para determinar la "calidad" o "eficacia" de los clasificadores entrenados. A continuación, una breve descripción de las mismas:

##### 2.4.3.1 Matriz de Confusión
***
A

##### 2.4.3.2 Precision, Recall, Fall-off, F-measure
***
A

##### 2.4.3.3 Medidas macro y micro
***
A

##### 2.4.3.4 Accuracy
***
A


## 3. Experimentación
***
En esta sección se detalla el conjunto de pruebas realizadas, agregandose observaciones pertinentes y comparaciones entre modelos.

### 3.1. Metodología
***

Con el objetivo de evaluar las estrategias planteadas anteriormente y su efectividad, se probaron combinaciones de las mismas frente a los dos modelos desarrollados, se encaró la exprimentación diviendola en las siguientes etapas:

1. **Configuraciones paramétricas:** Luego de llegar a un modelo relativamente libre de errores (al menos de errores identificados), se realizaron pruebas para múltiples configuraciones paramétricas y comparar cuales de ellas resultaron ser las mejores. En esta etapa se detallan los datos obtenidos. A causa de el tamaño del conjunto Covertype, no fue posible probar todas las configuraciones paramétricas para el mismo, se realizaron sólo las que teóricamente deberían dar un mejor resultado; este no fue el caso para Iris donde se probaron y juntaron datos de todas las configuraciones paramétricas.
<br><br>
2. **Elección y comparación de representantes:** A falta de suficientes corridas para generar agrupamientos de configuraciones, se compararon manualmente los resultados obtenidos en las pruebas de cada configuración paramétrica de la etapa anterior.

### 3.2. Recopilación y ajuste
***

Aca hablo yo de como llegamos al algoritmo base que funca

### 3.3. Configuraciones paramétricas
***
Acá hablo yo

#### 3.3.1. Evaluaciones
***
Acá hablo yo

##### 3.3.1.1 Conjuntos de datos
***
Iris y Covertype (2 opciones) Acá hablo yo

##### 3.3.1.2 Modelos
***
Árbol y Bosque (2 opciones)

##### 3.3.1.3 Parámetros
***
Fijo, Variable, C45 (3 opciones)
Ganancia, Ratio de Ganancia, Reducción de Impureza (3 opciones)
Acá hablo yo

#### 3.3.2. Resultados
***
Acá hablo yo

##### 3.3.3.1. Matrices de Confusión
***
Introducción

**Acá va el lennyscript de matrices**

##### 3.3.3.2. Comparación de resultados
***

Introducción

<table>
    <tr>
        <th>Iteraciones</th>
        <th>Turnos</th>
        <th>Ratio $\gamma$</th>
        <th>Pesos iniciales</th>
        <th>Pesos finales (aprox)</th>
        <th>Partidas ganadas</th>
        <th>Partidas perdidas</th>
        <th>Partidas empatadas</th>
        <th>Ratio de victorias</th>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.50, 0.00, 1.00, 0.20, 0.42]</td>
        <td>13%</td>
        <td>7%</td>
        <td>80%</td>
        <td>0,65</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.27, 0.00, 0.08, 0.91, 1.00]</td>
        <td>7%</td>
        <td>2%</td>
        <td>91%</td>
        <td>0,78</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.68, 0.24, 0.00, 1.00, 0.38]</td>
        <td>21%</td>
        <td>6%</td>
        <td>73%</td>
        <td>0,78</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.00, 1.00, 0.76, 0.69, 0.60]</td>
        <td>35%</td>
        <td>14%</td>
        <td>51%</td>
        <td>0,71</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.83, 1.00, 0.65, 0.64, 0.00]</td>
        <td>58%</td>
        <td>24%</td>
        <td>18%</td>
        <td>0,70</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.83, 1.00, 0.65, 0.64, 0.00]</td>
        <td>55%</td>
        <td>17%</td>
        <td>28%</td>
        <td>0,76</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.00, 0.56, 1.00, 0.36, 0.29]</td>
        <td>48%</td>
        <td>34%</td>
        <td>18%</td>
        <td>0,58</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.20, 0.68, 1.00, 0.32, 0.00]</td>
        <td>44%</td>
        <td>32%</td>
        <td>24%</td>
        <td>0,57</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.21, 0.74, 0.96, 1.00, 0.00]</td>
        <td>50%</td>
        <td>28%</td>
        <td>22%</td>
        <td>0,64</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.39, 0.80, 1.00, 0.53, 0.00]</td>
        <td>50%</td>
        <td>28%</td>
        <td>22%</td>
        <td>0,64</td>
    </tr>
    <tr style="background-color: #afdb99;">
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.00, 0.39, 1.00, 0.53, 0.18]</td>
        <td>55%</td>
        <td>17%</td>
        <td>28%</td>
        <td>0,76</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[1.00, 0.86, 0.96, 0.36, .00]</td>
        <td>31%</td>
        <td>6%</td>
        <td>63%</td>
        <td>0,83</td>
    </tr>
    <caption>Tabla 1 - Resultados de entrenamiento de <b>Instancia 1</b> para cada configuración paramétrica</caption>
</table>


### 3.4. Elección y comparación de representantes
***

#### 3.4.1. Elección
***
Acá hablamos del proceso para elegir a los representantes

Se adjunta script de graficas

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia1.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.17 - Partidas ganadas para cada modelo entrenado de la instancia 1</label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia2.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.18 - Partidas ganadas para cada modelo entrenado de la instancia 2</label>
    </div>
</div>

#### 3.4.2. Comparación
***
Aca hablamos de los mejores y los comparamos

**Observaciones generales:**
- La **instancia 2** alcanzó rendimientos más altos que la **instancia 1**
- Observando la gráfica, los modelos de la **instancia 1** se encuentran en condiciones parejas, excepto por aquellos entrenados utilizando $\gamma = 0.5$.
- Observando la gráfica, los "mejores" modelos de la **instancia 2** son aquellos con un peor ratio de victorias en la tabla 2 (compensado por su cantidad de partidas ganadas). Se determina que el peso de la cantidad de partidas ganadas es mayor al del ratio de victorias en este caso, ya que los otros ejemplos presentan una gran cantidad de empates.

**Interpretaciones:**
- La **instancia 1** es más robusta en relación a las distintas configuraciones paramétricas. La **instancia 2** en cambio es más sensible, devolviendo mejores resultados solo para el vector inicial con pesos pequeños. Esto puede implicar que si se entrenase más tiempo ambas instancias, la **instancia 1** alcance un mejor rendimiento que la **instancia 2**.
- Descartando el punto anterior, se interpreta que la separación de atributos sin normalizar (**instancia 2**) tiene un mejor rendimiento. Esto puede deberse a la pérdida de información al agrupar $A_i, B_i, C_i, D_i$ en $A, B, C, D$, al hecho de haber utilizado **normalización mín-máx** en los pesos, lo cual genera que uno sea siempre 0 y otro siempre 1, o a alguna otra razón no considerada.
- Tomando el hecho de que el vector con mejores resultados en la **instancia 2** es $w = [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]$, se puede interpretar que priorizar $A_1$ y $A_2$ por sobre el resto de pesos es una práctica acertada para este problema.

## 4. Conclusiones
***

#### 4.1. Respecto a los Modelos
***
Acá hablar sobre la demora de los bosques y la dificultad de establecer una votación que estuviera buena. La randomness al empatar es bastante mierda y mete ruido.

#### 4.2. Respecto a los Atributos
***
Acá hablar de que el one hot encoding no rinde pa arboles

#### 4.3. Respecto a los Parámetros
***
Acá hablar de que los arboles no encaran tanto en continuos, fixed es caca, c45 y variables son mejores pero para un dataset tan grande nos cagó.

#### 4.4. Respecto a los Resultados
***
Acá hablar de que covertype da peores resultados porque es un dataset muy grande. Más conclusiones viendo los resultados. Hablar de que los promedios ponderados nos dieron mejor info que los promedios generales porque hay mucha cantidad de b1 en covertype y poca de otros como b4.

#### 4.5. Posibles mejoras
***
A
- Usar todos los valores en variables y c4.5
- Usar distintas measures en c4.5
- Mantener el onehotencoding o al menos no tratar como continuos esos atributos
